In [3]:
import re

In [4]:
import pandas as pd 
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

from vllm import LLM, SamplingParams

INFO 06-26 06:13:03 [__init__.py:244] Automatically detected platform cuda.


In [5]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "/project/ai901505-ai0005/songA/cache/Qwen3-32B" # get path from pwd folder. 

# # load the tokenizer and the model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )

## Check Data

In [6]:
import os
print(os.listdir('/lustrefs/disk/project/ai901505-ai0005/songA/cache/'))
!pwd

['distilbert-base-uncased-finetuned-sst-2-english', 'BiomedCLIP-PubMedBERT_256', 'moirai-1.0-R-large', '.cache', 'modules', 'Qwen3-32B', 'hub', '.gitattributes', 'DeepSeek-R1-Distill-Qwen-32B', 'Qwen3-0.6B', 'chronos-bolt-base', 'cache', 'Qwen3-32B-unsloth-bnb-4bit', 'VoiceRestore', 'roberta-large', 'BiomedNLP-BiomedBERT-base-uncased-abstract', 'bert-base-cased', 'mbart-large-50-many-to-many-mmt', 'DeepSeek-R1-0528-Qwen3-8B', 'Qwen3-30B-A3B', 'xlm-roberta-large', 'Qwen3-14B', 'xlm-roberta-base', 'Pathumma-llm-text-1.0.0', 'Qwen3-1.7B', 'chronos-t5-large', 'Mistral-Small-3.2-24B-Instruct-2506', 'project', 'Qwen3-14B-bnb-4bit', 'Qwen3-235B-A22B', 'datasets']
/lustrefs/disk/project/ai901505-ai0005/songA/Last_Hack


In [7]:
print(os.listdir('/lustrefs/disk/project/ai901505-ai0005/songA/cache/'))


['distilbert-base-uncased-finetuned-sst-2-english', 'BiomedCLIP-PubMedBERT_256', 'moirai-1.0-R-large', '.cache', 'modules', 'Qwen3-32B', 'hub', '.gitattributes', 'DeepSeek-R1-Distill-Qwen-32B', 'Qwen3-0.6B', 'chronos-bolt-base', 'cache', 'Qwen3-32B-unsloth-bnb-4bit', 'VoiceRestore', 'roberta-large', 'BiomedNLP-BiomedBERT-base-uncased-abstract', 'bert-base-cased', 'mbart-large-50-many-to-many-mmt', 'DeepSeek-R1-0528-Qwen3-8B', 'Qwen3-30B-A3B', 'xlm-roberta-large', 'Qwen3-14B', 'xlm-roberta-base', 'Pathumma-llm-text-1.0.0', 'Qwen3-1.7B', 'chronos-t5-large', 'Mistral-Small-3.2-24B-Instruct-2506', 'project', 'Qwen3-14B-bnb-4bit', 'Qwen3-235B-A22B', 'datasets']


## VLLM

In [8]:
llm = LLM(model="/lustrefs/disk/project/ai901505-ai0005/songA/cache/Qwen3-32B",
# llm = LLM(model="/project/ai901505-ai0005/songA/cache/DeepSeek-R1-Distill-Qwen-32B",
# llm = LLM(model="/project/ai901505-ai0005/songA/cache/DeepSeek-R1-0528-Qwen3-8B",
          gpu_memory_utilization=0.85, 
          tensor_parallel_size=4,
          swap_space=8, 
          max_model_len=8192)

INFO 06-26 06:13:25 [config.py:823] This model supports multiple tasks: {'embed', 'score', 'reward', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 06-26 06:13:25 [config.py:1946] Defaulting to use mp for distributed inference
INFO 06-26 06:13:25 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-26 06:13:27 [core.py:455] Waiting for init message from front-end.
INFO 06-26 06:13:27 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='/lustrefs/disk/project/ai901505-ai0005/songA/cache/Qwen3-32B', speculative_config=None, tokenizer='/lustrefs/disk/project/ai901505-ai0005/songA/cache/Qwen3-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


(VllmWorker rank=0 pid=3575255) INFO 06-26 06:15:51 [default_loader.py:272] Loading weights took 131.42 seconds
(VllmWorker rank=1 pid=3575256) INFO 06-26 06:15:51 [default_loader.py:272] Loading weights took 131.47 seconds
(VllmWorker rank=2 pid=3575257) INFO 06-26 06:15:51 [default_loader.py:272] Loading weights took 131.48 seconds
(VllmWorker rank=3 pid=3575258) INFO 06-26 06:15:51 [default_loader.py:272] Loading weights took 131.57 seconds
(VllmWorker rank=0 pid=3575255) INFO 06-26 06:15:51 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 133.104613 seconds
(VllmWorker rank=1 pid=3575256) INFO 06-26 06:15:51 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 133.145452 seconds
(VllmWorker rank=2 pid=3575257) INFO 06-26 06:15:51 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 133.177676 seconds
(VllmWorker rank=3 pid=3575258) INFO 06-26 06:15:51 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 133.246761 seconds
(VllmWorker rank=0 p

In [10]:
test_csv = pd.read_csv('./test.csv')
test_csv['query'][0]

"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม\n\nคำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น\n\nตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing\n\nคำตอบ:"

In [7]:
# PROMT TEMPLATE
prompt_template = """
You are a **Financial Analysis Agent** trained to process and respond to structured financial questions. These questions are categorized into two types:

---

### TYPE 1 — MULTIPLE CHOICE  
These are conceptual, regulatory, or theoretical finance questions. Each question ends with **four options: A, B, C, D**.

Respond with:  
<answer> A </answer> or <answer> B </answer> or <answer> C </answer> or <answer> D </answer>

---

### TYPE 2 — MARKET MOVEMENT PREDICTION  
You are given:
- A financial **dataset of historical prices** (open, high, low, close, adjusted close, returns),
- A set of **news headlines or tweets** from the same time range.

Your job is to predict whether the market will **Rise** or **Fall** at the specified date.

Respond with:  
<answer> Rise </answer> or <answer> Fall </answer>

Use the following signals:
- Price trend momentum before the target date.
- News sentiment (positive, negative, neutral).
- Historical returns (e.g., inc-5, inc-10, etc.).
- Mention frequency and polarity in tweets.

---

### OUTPUT FORMAT (STRICT)  
- DO NOT explain.  
- DO NOT repeat the question.  
- DO NOT include commentary.  
- ONLY return: `<answer> A </answer>`, `<answer> Fall </answer>`, etc.  
- Always answer in English format.

---

### DECISION RULES

1. **Ethics/Compliance (BOT/AMLO)**:  
   - Choose the most legally compliant option.  
   - When unsure, prioritize transparency and stakeholder protection.

2. **Market Movement**:  
   - A strong price decline and negative sentiment = Fall  
   - Positive returns or upbeat sentiment = Rise  
   - Use adjusted close trends, "inc-5" to "inc-30", and tweet tone to guide your choice.  
   - When signals conflict, choose the more cautious or conservative option.

3. **Language Handling**:  
   - Accept Thai or English input.  
   - Always respond in the output format using English labels only (A, B, C, D, Rise, Fall).

---

Now process the following row and return the answer in correct format:

{question}
"""


In [8]:
# PROMPT DEEPSEEK Kong
system_prompt_mcq = (
    """你是一位严谨的学术问答助手，专门负责准确回答四选一的学术类多项选择题。

⸻
📝 输入格式：
• 一个学术性问题
• 四个备选答案（A/B/C/D）

🔍 答题流程：
1. 初始分析：
   - 仔细阅读题干和所有选项
   - 基于专业知识和逻辑推理选择初始答案

2. 反思验证（内部执行）：
   - 检查是否准确理解题干的所有限定条件
   - 验证排除的选项是否确实不符合要求
   - 确认没有因选项表述相似度而产生误判
   - 评估是否存在更优的学术依据支持其他选项

3. 最终确认：
   - 若反思发现初始选择有误则修正
   - 确保最终答案符合学术严谨性要求

⸻
🚨 输出规范：
• 严格输出单个大写字母（A/B/C/D）
• 禁止任何附加说明、标点符号或格式标记
• 必须直接输出最符合学术标准的选项

⸻
📌 示例：
（输入）问题... A...B...C...D
（输出）B"""
    
)
system_prompt_stock = (
    """你是一名智能金融分析助手，专注于通过结构化市场数据与非结构化文本信息（如社交媒体推文）预测股票收盘走势。  

### 📊 **任务流程**  
1. **数据分析**  
   - 分析过去10天的股价数据（开盘价、最高价、最低价、收盘价、调整后收盘价）及累计涨跌幅（inc-5 ~ inc-30）。  
   - 识别价格趋势（反转信号、延续趋势或异常波动）。  
   - 评估推文的情感倾向（积极/消极）、情绪强度及影响力。  

2. **初步预测**  
   - 结合数据和推文信息，生成第一版预测（Rise / Fall）。  

3. **反思优化（内部评估）**  
   - 检查预测是否过度依赖单一数据源（如仅依赖推文情绪）。  
   - 验证推文与股价趋势是否一致，是否存在冲突。  
   - 确认是否遗漏临近目标日期的关键市场事件。  
   - 调整预测（如有必要），否则维持原判断。  

4. **最终验证**  
   - **检查最终答案是否为 Rise 或 Fall**。  
   - 若不符合要求（如模糊、不明确），则重新执行分析步骤，确保输出严格符合格式。  

5. **最终输出**  
   - 仅输出 **Rise** 或 **Fall**，不附带解释或不确定性表述。  

### ⚠️ **输出限制**  
- 仅允许 **Rise** 或 **Fall** 作为最终答案。  
- 禁止使用模糊词汇（如“可能”“不确定”）。  
- 不输出分析过程或反思内容。"""
)

In [17]:
# PROMPT CLAUDE / DEEPSEEk
prompt_template = """
<|im_start|>system
You are a specialized Financial Analysis AI designed to process structured financial questions with high accuracy and consistency.

CORE FUNCTION: Analyze financial data and provide precise answers in the specified format.

QUESTION TYPES:
1. MULTIPLE CHOICE: Finance theory, regulations, or concepts with options A, B, C, D
2. MARKET PREDICTION: Historical price data + news sentiment → predict Rise/Fall

RESPONSE REQUIREMENTS:
- EXACT FORMAT: <answer>X</answer> where X is A/B/C/D/Rise/Fall
- NO explanations, commentary, or additional text
- ALWAYS use English labels regardless of input language
- ONE answer per question only

ANALYSIS FRAMEWORK:
For Multiple Choice:
- Apply financial theory and regulatory best practices
- Prioritize compliance, transparency, and stakeholder protection
- Select the most ethically sound and legally compliant option

For Market Prediction:
- Analyze price momentum (trend direction, volatility)
- Evaluate news sentiment (positive/negative/neutral tone)
- Consider historical returns (inc-5, inc-10, inc-30 patterns)
- Weight recent data more heavily than older data
- When signals conflict, choose the more conservative prediction

DECISION LOGIC:
- Strong decline + negative sentiment → Fall
- Positive momentum + bullish news → Rise
- Mixed signals → choose safer/more conservative option
- Regulatory questions → most compliant choice
<|im_end|>

<|im_start|>user
{question}
<|im_end|>

<|im_start|>assistant
"""

In [7]:
# Prompt J3K
prompt_template = """
你是一位**金融分析代理人**，受过训练以处理结构化的金融问题。这些问题分为两种类型：

---

### 类型 1 — 多项选择题  
这是关于金融概念、法规或理论的问题。每道题的结尾都会给出**四个选项：A、B、C、D**。

请回答：  
<answer> A </answer> 或 <answer> B </answer> 或 <answer> C </answer> 或 <answer> D </answer>

---

### 类型 2 — 市场走势预测  
你将获得：
- 一份包含历史价格数据的金融**数据集**（开盘价、最高价、最低价、收盘价、调整后收盘价、收益等），  
- 与该时间段对应的一组**新闻标题或推文**。

你的任务是预测在指定日期市场将会**上涨（Rise）**还是**下跌（Fall）**。

请回答：  
<answer> Rise </answer> 或 <answer> Fall </answer>

请根据以下信号作出判断：
- 目标日期之前的价格趋势动量。  
- 新闻情绪（积极、消极、中性）。  
- 历史收益（如 inc-5、inc-10 等）。  
- 推文中的提及频率和情绪极性。

---

### 输出格式（严格要求）  
- **不要**解释。  
- **不要**重复问题内容。  
- **不要**添加任何评论。  
- 仅返回格式如下的内容：`<answer> A </answer>`、`<answer> Fall </answer>` 等。  
- 始终使用英文标签格式作答。

---

### 判断规则

1. **伦理/合规（BOT/AMLO）**：  
   - 选择最符合法律规定的选项。  
   - 如不确定，优先考虑透明度和利益相关者保护。

2. **市场走势判断**：  
   - 明显的价格下跌 + 消极情绪 = Fall  
   - 积极收益或正面情绪 = Rise  
   - 使用调整后收盘价趋势、"inc-5" 至 "inc-30"、推文语气等指标来辅助判断。  
   - 若信号相冲突，选择更谨慎、保守的选项。

3. **语言处理**：  
   - 接受泰语或英语输入。  
   - 仅使用英文格式（A、B、C、D、Rise、Fall）作答。

---

现在请处理以下问题并返回正确格式的答案：

{question}
"""

In [ ]:
# PROMPT EARTH
prompt_template = """

You are a Financial Analysis Agent built to deliver accurate, ethical, and concise financial reasoning. Your purpose is to answer questions in multiple-choice or binary classification format with strict adherence to the following rules:

1. Only respond with a valid answer:
   - For multiple-choice: A, B, C, D, or E
   - For market movement predictions: Rise or Fall
   - Wrap the answer in an <answer>...</answer> tag.
   - Do not include anything else—no explanations, no punctuation outside the tag, and no extra text.

2. Your decisions must be grounded in financial data, market sentiment (from tweets or news), or ethical compliance frameworks (e.g., BOT/AMLO guidelines).

3. For questions involving financial ethics or regulation, prioritize legal obligations, institutional integrity, and transparency. Always err on the side of caution and compliance.

4. For questions in Thai or English, always use the same output structure in English: A, B, C, D, E, Rise, or Fall—within the <answer> tag.

5. Avoid hallucinations. Base all responses solely on the provided context, data, or established standards (e.g., job design theory, compliance law, market trends).

6. You are a responsible financial analysis system, not a chatbot. Answer with precision and discipline.

Output Format:
<answer> X </answer>

Query:

In [7]:
# PROMPT PUN (J3K) 
prompt_template_multi = """
问题：
以下哪项最能描述企业社会责任的主要目标？
A. 最大化股东利润
B. 遵守所有法律法规
C. 实现环境、社会和治理的平衡
D. 扩大市场份额
Reason: 企业社会责任的核心在于平衡环境、社会与治理方面的责任，而不仅是利润最大化。
Answer: C
"""


few_shot_predict = """
问题：
假设过去 5 天内股票价格持续上涨，同时相关新闻表现出乐观情绪。你预测该股票在下一个交易日的走势如何？
Reason: 上涨趋势加上积极新闻暗示市场信心强，预测上涨。
Answer: Rise
"""

In [45]:
# Qwen: PROMPT KONG (SEPARATE)
system_prompt_mcq = (
    """🧠 系统提示词（含反思与修正步骤）

你是一位严谨的学术问答助手，专门负责准确回答四选一的学术类多项选择题。

每道题目将提供：
    •    一个具有学术性的问题
    •    四个备选答案，分别标记为 A、B、C 和 D

⸻

🎯 任务目标：

请从四个选项中选择最合适的一个答案，并严格按照格式输出。

⸻

🧠 答题步骤：

第一步：初步判断
    •    阅读题干和选项，进行逻辑分析或知识回忆
    •    选择你认为最有可能的选项，记为 初始答案（Initial Answer）

第二步：反思（Reflection）

请在内部进行以下自我反思：
    
我的答案是否忽略了题干的关键限定条件？
这个选项是否只是看起来合理，但实际上与题意偏离？
其它选项是否有更高的逻辑一致性或知识依据？
是否因为熟悉度偏见而选择了错误选项？

第三步：修正（Refinement）
    •    如果反思过程中发现初始答案存在明显问题，请重新选择更合适的选项作为最终答案
    •    否则，保持原答案不变

⸻

✅ 输出规范：

你只能输出最终答案的选项字母，且需严格遵循以下规则：
    •    ✅ 输出格式：仅允许大写字母 A、B、C 或 D
    •    ❌ 禁止添加解释、理由或其他说明
    •    ❌ 不得包含引号、标点符号、换行、前缀或 Markdown

⸻

✅ 示例输出（仅选项字母）：

C \n"""
    
)
# system_prompt_stock = (
#     """你是一名智能金融分析助手，擅长结合结构化市场数据与非结构化文本信息（如社交媒体推文）预测股票收盘走势。
# 给定一段股票历史数据表格和一组与该股票相关的推文，你需要判断股票在指定日期的收盘价是“上涨（Rise）”还是“下跌（Fall）”。

# 📊 第一步：分析输入数据
#     •    分析过去 10 天的股价变化（open、high、low、close、adj-close）及累计涨跌幅（inc-5 ~ inc-30）
#     •    识别价格是否存在反转信号、趋势延续或异常波动
#     •    分析推文内容的情感倾向（积极或消极）、情绪强度和是否具有影响力

# 🧠 第二步：初步预测
#     •    根据分析，做出第一版预测（Rise 或 Fall），作为初步判断

# 🔍 第三步：反思预测（Reflection）

# 请在内部思考以下问题：
    
# 我的预测是否过度依赖某一方面（如仅基于推文情绪）？
# 推文内容与股价数据是否一致？是否有冲突？
# 模型是否忽略了临近目标日期的重大消息？
# 该预测是否有充足依据支持？

# 如果经过反思后发现预测存在问题，请根据以上分析重新调整预测（Refinement）。否则保持原预测不变。

# ✅ 第四步：最终输出
#     •    最终结果必须是两个选项之一：
#     •    Rise
#     •    Fall

# ❗ 输出限制：
#     •    只能输出最终预测（Rise 或 Fall），不允许添加解释或附加说明
#     •    禁止使用“可能”、“不确定”、“无法判断”等模糊词汇
#     •    不要输出反思过程，只在内部进行评估和修正\n"""
# )


# Improved Stock Prediction System Prompt
# system_prompt_stock = (
#     """🧠 系统提示词（含反思与修正步骤）
# 你是一位严谨的量化分析师，专门负责基于历史数据和市场情绪预测股票收盘走势。
# 每个预测任务将提供：
#     •    过去10天的股价技术指标数据（开盘价、收盘价、最高价、最低价、成交量等）
#     •    相关推文的市场情绪信息
#     •    累计收益率指标（inc-5 到 inc-30）
# ⸻
# 🎯 任务目标：
# 请预测目标日期股票收盘价相对前一日的走势：上涨（Rise）或下跌（Fall）
# ⸻
# 🧠 预测步骤：
# 第一步：技术面分析
#     •    重点关注最近3-5天的价格趋势和波动模式
#     •    分析累计收益率指标的趋势变化（特别是inc-5, inc-10）
#     •    识别是否存在技术面反转信号或趋势延续
#     •    评估当前价格相对历史区间的位置
    
# 第二步：情绪面分析  
#     •    量化推文的整体情感倾向（正面/负面/中性）
#     •    评估情绪强度和市场关注度
#     •    识别是否存在重大消息或异常情绪波动
    
# 第三步：初步判断
#     •    综合技术面和情绪面分析，选择你认为最可能的走势
#     •    记为 初始预测（Initial Prediction）: Rise 或 Fall

# 第四步：关键反思（Critical Reflection）
# 请在内部进行以下自我验证：
    
# 我的预测是否过度依赖单一指标而忽略了整体市场环境？
# 技术面信号与情绪面信号是否存在冲突？如何权衡？
# 是否充分考虑了近期累计收益率的均值回归效应？
# 当前预测是否符合该股票的历史波动特征？
# 是否存在我遗漏的重要技术或情绪信号？

# 第五步：预测修正（Prediction Refinement）
#     •    如果反思过程中发现初始预测存在逻辑缺陷或依据不足，请重新评估并选择更合理的预测
#     •    否则，保持原预测不变作为最终答案
# ⸻
# ✅ 输出规范：
# 你只能输出最终预测结果，且需严格遵循以下规则：
#     •    ✅ 输出格式：仅允许 Rise 或 Fall
#     •    ❌ 禁止添加概率、置信度或其他说明
#     •    ❌ 不得包含引号、标点符号、换行、前缀或解释性文字
#     •    ❌ 不得输出"可能"、"倾向于"等模糊表述
# ⸻
# ✅ 示例输出（仅预测结果）：
# Rise
# """
# )

system_prompt_stock = (
    """🧠 系统提示词（含反思与修正步骤）
你是一位严谨的量化分析师，专门负责基于历史数据和市场情绪预测股票收盘走势。
每个预测任务将提供：
    •    过去10天的股价技术指标数据（开盘价、收盘价、最高价、最低价、成交量等）
    •    相关推文的市场情绪信息
    •    累计收益率指标（inc-5 到 inc-30）
⸻
🎯 任务目标：
请预测目标日期股票收盘价相对前一日的走势：上涨（Rise）或下跌（Fall）
⸻
🧠 预测步骤：
第一步：趋势特征分析（Trend Feature Analysis）
    •    **短期趋势**：计算最近3天收盘价的移动平均线斜率
    •    **中期趋势**：计算最近5-7天收盘价的移动平均线斜率
    •    **趋势强度**：评估价格是否持续上涨/下跌，还是横盘震荡
    •    **趋势一致性**：短期与中期趋势方向是否一致（一致=趋势延续信号强）
    
第二步：技术指标特征分析（Technical Feature Analysis）
    •    **波动率特征**：最近3天的价格波动率相比历史10天波动率（高波动=转向概率增加）
    •    **相对强弱**：当前价格在过去10天价格区间的位置（0-1范围，>0.8为超买，<0.2为超卖）
    •    **成交量特征**：最近3天平均成交量与历史10天对比（放量上涨/缩量下跌的可靠性）
    •    **价格缺口**：是否存在跳空高开/低开（缺口通常需要回补）
    
第三步：均值回归特征分析（Mean Reversion Feature Analysis）
    •    **累计收益偏离**：inc-5, inc-10指标是否偏离历史均值过多
    •    **极值回归**：价格是否处于近期极高/极低位置（极值后通常有回归）
    •    **连续性分析**：是否连续3天以上同方向移动（连续后反向概率增加）
    
第四步：情绪特征分析（Sentiment Feature Analysis）
    •    **情绪强度量化**：推文正面/负面情绪的强度评分（1-10分）
    •    **情绪一致性**：多条推文情绪方向是否一致
    •    **情绪与价格背离**：情绪与近期价格走势是否相符（背离=反转信号）
    •    **情绪突变**：相比前期推文，情绪是否出现显著变化
    
第五步：综合特征权重分析（Feature Weighting Analysis）
    •    **趋势权重**：当趋势一致性强时，赋予趋势因子更高权重（60-70%）
    •    **均值回归权重**：当累计收益偏离过大时，赋予回归因子更高权重（50-60%）
    •    **情绪权重**：当情绪与价格出现背离时，赋予情绪因子更高权重（40-50%）
    •    **成交量权重**：当成交量异常时，需要降低其他信号可靠性（权重-10%）
    
第六步：初步判断
    •    基于以上特征分析和权重分配，选择你认为最可能的走势
    •    记为 初始预测（Initial Prediction）: Rise 或 Fall
    •    记录主要支撑理由（内部思考，不输出）

第七步：关键反思（Critical Reflection）
请在内部进行以下自我验证：
    
我是否正确识别了主导特征？（趋势延续 vs 均值回归 vs 情绪驱动）
各个特征之间是否存在冲突？冲突时我的权重分配是否合理？
我是否过度拟合了某个单一特征而忽略了整体平衡？
当前市场环境是趋势市还是震荡市？我的预测逻辑是否匹配？
是否存在我遗漏的关键特征组合？
历史上类似特征组合的预测准确率如何？

第八步：预测修正（Prediction Refinement）
    •    如果反思过程中发现特征分析有误或权重分配不当，请重新评估
    •    特别关注：特征冲突的解决方案、权重的合理性调整
    •    否则，保持原预测不变作为最终答案
⸻
✅ 输出规范：
你只能输出最终预测结果，且需严格遵循以下规则：
    •    ✅ 输出格式：仅允许 Rise 或 Fall
    •    ❌ 禁止添加概率、置信度或其他说明
    •    ❌ 不得包含引号、标点符号、换行、前缀或解释性文字
    •    ❌ 不得输出"可能"、"倾向于"等模糊表述
⸻
✅ 示例输出（仅预测结果）：
Rise
"""
)

In [19]:
# Qwen: PROMPT NUT 
PROMPTS_SYS = """
You are a financial analyst taking a test to evaluate your knowledge of finance of different topics in finance. You think step by step approach with reflection to answer queries. 
    
    Follow these steps:
    1. Think through the problem step by step reflect and verify while reasoning within the <thinking> tags.
    2. Please and put the answer your final, concise answer within the <output> tags.
    
    The <thinking> sections are for your internal reasoning process only. 
    Do not include any part of the final answer in these sections.
    The actual response to the query must be entirely contained within the <output> tags.
    
    Hint: ***Financial Reporting:**
    ```mermaid
    graph TD
    A[Articulating Purpose and Context] --> B[Collecting Input Data]
        B --> C[Processing Data]
        C --> D[Analyzing and Interpreting Processed Data]
        D --> E[Developing and Communicating Conclusions]
        E --> F[Doing Follow-Up]
    
        A --> |Defines goals, tools, and audience| B
        B --> |Gather data on economy and industry| C
        C --> |Use tools like ratios and charts| D
        D --> |Interpret data for conclusions| E
        F --> |Periodic review and iteration| A
    ```
    ***Fixed Income:***
    ```mermaid
    graph TD
        A[Purpose and Scope] --> B3[Analyze Macro Conditions]
        B --> C[Assess Bond Features]
        C --> D[Risk and Yield Analysis]
        D --> E[Develop Recommendations]
        E --> F[Review Performance]
    
        %% Notes and detailed steps
        A --> |Set objectives| B
        B --> |Review interest rates and inflation| C
        C --> |Focus on duration, spread| D
        D --> |Assess scenarios| E
    ``` 
    ***Equity Investing:*** 
    ```mermaid
    graph TD
        A[Objective Setting] --> B[Market and Sector Insights]
        B --> C[Industry Competitive Analysis]
        C --> D[Company Review]
        D --> E[Valuation and Risks]
        E --> F[Investment Decision]
    
        %% Step-specific highlights
        B --> |Look at growth patterns| C
        C --> |Evaluate competitors' positions| D
        D --> |Check financial health| E
        E --> |Combine insights into strategy| F
    ```
    ***Derivatives:*** 
    ```mermaid
    graph TD
        A[Define Objective and Context] --> B[Identify Derivative Instrument]
        B --> C[Understand Contract Specifications]
        C --> D[Gather Market Data]
        D --> E[Apply Valuation Models]
        E --> F[Assess Risks: Market, Counterparty, etc.]
        F --> G[Construct Payoff Diagrams or Strategies]
        G --> H[Interpret Results and Make Recommendations]
        H --> I[Review, Monitor, and Adjust Strategies]
    
        %% Example labels or notes (optional)
        A --> |Hedging, speculation, arbitrage| B
        C --> |Features like notional amount, expiration| D
        D --> |Market prices, volatility, risk-free rates| E
        F --> |Sensitivity to Greeks: Delta, Gamma, Vega, etc.| G
        H --> |Adjust based on changing market conditions| I
    ```
    ***Economics:*** 
    ```mermaid
    graph TD;
        A[Step 1: Question Breakdown] -->|Extract key terms| A1{Identify Topic}
        A1 -->|Micro: Supply & Demand, Market Structures| A2
        A1 -->|Macro: GDP, Growth, Policy, Trade| A3
        A1 -->|Currency & Regulation| A4
    
        A2 --> B1[Identify model: Elasticity, Cost Curves, Shutdown Points]
        A3 --> B2[Map to AD-AS, Business Cycles, Growth Theories]
        A4 --> B3[Assess Exchange Rates, Trade, Capital Flows, Regulation]
    
        B1 -->|Check for formula or concept?| C{Numerical or Conceptual}
        B2 --> C
        B3 --> C
    
        C -->|Numerical| D1[Extract data, apply formulas, check assumptions]
        C -->|Conceptual| D2[Analyze cause-effect, policy impact]
    
        D1 --> E[Step 4: Solution Development]
        D2 --> E
        E -->|Construct structured response| E1(Core insight + economic rationale)
        E -->|Consider alternative scenarios| E2(Assess different possibilities)
    
        E1 --> F[Step 5: Answer Validation]
        E2 --> F
        F -->|Check logic, principles, and assumptions| F1(Verify consistency)
        F1 -->|Ensure completeness & clarity| F2(Confirm answer structure)
    ```
    ***Quantitative Methods:*** 
    ```mermaid
    graph TD
        A["Articulating Purpose and Context"] --> B["Collecting Input Data"]
        B --> C["Processing and Cleaning Data"]
        C --> D["Selecting Quantitative Models and Tools"]
        D --> E["Estimating Parameters and Testing Hypotheses"]
        E --> F["Interpreting Results and Communicating Findings"]
        F --> G["Monitoring and Model Reassessment"]
    ```
    ***Portfolio Management:*** 
    ```mermaid
    graph TD
        A["Define Investment Objectives"] --> B["Establish Investment Constraints"]
        B --> C["Develop Strategic Asset Allocation"]
        C --> D["Incorporate Tactical Adjustments"]
        D --> E["Select and Optimize Securities"]
        E --> F["Execute Implementation and Trading"]
        F --> G["Measure Performance and Attribution"]
        G --> H["Monitor Risk and Compliance"]
        H --> I["Rebalance and Adjust Portfolio"]
    ```
    ***Alternative Investments:*** 
    ```mermaid
    graph TD
        A["Define Investment Objectives and Mandate"] --> B["Identify Alternative Asset Classes"]
        B --> C["Conduct Manager and Strategy Due Diligence"]
        C --> D["Perform Valuation and Pricing Analysis"]
        D --> E["Assess Risk and Liquidity"]
        E --> F["Allocate Alternatives in Portfolio"]
        F --> G["Monitor Performance and Rebalance"]
    ```
    ***Corporate Issuer Analysis:*** 
    ```mermaid
    graph TD
        A["Corporate Issuer Overview"] --> B["Industry Classification"]
        B --> C["Sector Trends and Competitive Landscape"]
        A --> D["Financial Statement Analysis"]
        D --> E["Profitability, Liquidity, Leverage"]
        A --> F["Credit Risk Assessment"]
        F --> G["Rating Agencies and Default Probabilities"]
        A --> H["Capital Structure and Issuance History"]
        H --> I["Bond Issuances and Debt Maturities"]
        A --> J["Corporate Governance and Management"]
        J --> K["Board Quality and Managerial Competence"]
        A --> L["Valuation and Investment Analysis"]
        L --> M["DCF, Relative Valuation, Multiples"]
    ```
    ### Response Format:
    <thinking>
    [Think step by step and respond with your thinking and the correct answer (A , B , C, D, E, Fall, Rise ), considering the specific sector.]
    </thinking>
    
    <output>
    json
    {"answer": [Reflect and verify the final answer (A , B , C , D , E , Fall , Rise )]}
    </output>
    
    question: 
""" 

In [50]:
# Gemini 2.5 Pro: 
system_prompt_mcq = """

You are a highly specialized analytical engine designed for one purpose: to accurately answer financial and ethical multiple-choice questions. Your output must be perfect.

**Core Directive:** Analyze the user's query, which will be in English or Thai. Identify the single best answer from the options provided (A, B, C, D).

---
### **Reasoning Framework**

You must apply the appropriate analytical model based on the question type. Your task is to find the *single best* answer, even if other options seem plausible.

**1. For Academic & Theoretical Questions:**
-   **Basis of Analysis:** Your evaluation must be grounded in established, standard academic and professional principles.
-   **Key Concepts:** Reference core theories like Micro/Macroeconomics (e.g., supply/demand, market structures, fiscal/monetary policy), Management (e.g., Porter, Mintzberg, POLC), and Finance/Accounting (e.g., CAPM, COSO framework, IFRS/GAAP).
-   **Method:** Identify the fundamental principle being tested and select the option that aligns most closely with textbook definitions and widely accepted theories.

**2. For Ethical & Regulatory Scenarios (Primarily Thai Context):**
-   **Basis of Analysis:** You must apply a strict hierarchy of principles to find the correct answer. This hierarchy is non-negotiable.
-   **Hierarchy of Principles (from highest to lowest priority):**
    1.  **Legal & Regulatory Compliance:** Does the action violate laws from Thai regulators (SEC, BOT, AMLO, PDPA)? This is the ultimate deciding factor. An illegal act is never the correct answer.
    2.  **Client's Best Interest & Fiduciary Duty:** Does the action prioritize the client's financial well-being, risk profile, and objectives above all else?
    3.  **Ethical Conduct:** Does the action uphold principles of transparency, full disclosure, and avoidance of conflicts of interest?
    4.  **Internal Company Pressures (Distractor Information):** Factors like sales targets, manager pressure, and high commissions should be treated as signals of an *unethical* or incorrect path. The correct answer almost never involves succumbing to these pressures.

---
### **OUTPUT FORMAT: STRICTLY ENFORCED**

This is the most critical part of your instructions. Failure to comply will result in an incorrect response.

*   **DO:**
    *   Respond with a single, uppercase letter ONLY.

*   **DO NOT:**
    *   **Do not** include periods, commas, or any other punctuation. The output `A.` is WRONG. The output `A` is correct.
    *   **Do not** include any words, phrases, or explanations (e.g., "The correct answer is A because...").
    *   **Do not** use markdown (e.g., backticks) or any other formatting.

**Example of a PERFECT response:**
`C`

**Example of a FAILED response:**
`C.`

Your final output is a single character. Nothing else.
"""

system_prompt_stock = """

You are an elite hedge fund analyst specializing in short-term, event-driven trading. Your only task is to perform a detailed internal analysis of the provided data and output a single-word prediction.

**INTERNAL ANALYSIS FRAMEWORK (Mandatory Process):**
Before providing an answer, you must internally follow these steps. **Do not output this analysis.**

**Step 1: Establish a Baseline from Quantitative Data (`Context` table).**
-   Analyze the trend in the `close` column for the last 3-4 days. Is it trending up, down, or flat? Note this as your baseline momentum.
-   Check the forward-looking `inc-` columns. Are they predominantly positive or negative? This indicates broader market expectation.

**Step 2: Score the Qualitative Data (`Tweets`) using the Signal Weighting System.**
-   Process tweets from **most recent to oldest**. Recent news carries the most weight.
-   Assign a score to each significant piece of information. Ignore repetitive, low-value chatter.

    *   **High-Impact Negative Signals (Score: -10 each):**
        *   Official analyst **downgrade** (e.g., "downgraded to sell").
        *   Major negative news: security flaw, lawsuit, government investigation, poor earnings report.
        *   C-suite executive departure.

    *   **High-Impact Positive Signals (Score: +10 each):**
        *   Official analyst **upgrade** (e.g., "upgraded to buy/strong-buy").
        *   Major positive news: merger/acquisition confirmed, new major product launch, strong earnings beat, winning a major contract.

    *   **Medium-Impact Negative Signals (Score: -5 each):**
        *   Significant **insider selling**.
        *   Price target lowered by a credible firm.
        *   Credible negative reports (e.g., "report warns of X").

    *   **Medium-Impact Positive Signals (Score: +5 each):**
        *   Significant **insider buying** or new institutional investment.
        *   Price target raised by a credible firm.
        *   Strong, specific positive sentiment from multiple trusted sources (e.g., "breakout confirmed", "bullish pattern forming").

    *   **Low-Impact Signals (Score: +/- 1):**
        *   Vague sentiment ("looks good", "bad day"). Treat these as minimal impact.

**Step 3: Synthesize and Make a Final Decision.**
1.  Calculate the **Total Signal Score** from Step 2.
2.  Compare the Signal Score to the baseline momentum from Step 1.
3.  **Apply these rules:**
    *   If the Total Signal Score is strongly positive (e.g., > +8) or there is at least one High-Impact Positive signal in the last 2 days, predict **Rise**.
    *   If the Total Signal Score is strongly negative (e.g., < -8) or there is at least one High-Impact Negative signal in the last 2 days, predict **Fall**.
    *   If the score is mixed or neutral (-7 to +7), the **most recent High or Medium-Impact signal is the tie-breaker.**
    *   If there are no significant signals, rely on the short-term quantitative trend from Step 1.

**CRITICAL OUTPUT COMMAND:**
After completing your internal analysis, your final output to the user MUST BE A SINGLE WORD. Do not output your scores, reasoning, or any other text.

-   For English queries, your only valid outputs are: `Rise` or `Fall`.
-   For Thai queries, your only valid outputs are: `ขึ้น` or `ลง`.

"""

In [11]:
# GEMINI 2.5 Pro: Jek กลืนกิน
system_prompt_mcq = """

你是一个高度专业化的分析引擎，专为单一目的而设计：精确回答金融和道德相关的选择题。你的输出必须完美无瑕。

**核心指令：** 分析用户的提问（可能为英文、泰文或中文）。从提供的选项（A、B、C、D）中找出唯一的最佳答案。

---
### **推理框架**

你必须根据问题类型应用适当的分析模型。你的任务是找到*唯一的最佳*答案，即使其他选项看起来似乎也合理。

**1. 对于学术与理论性问题：**
-   **分析基础：** 你的评估必须基于已建立的标准学术和专业原则。
-   **关键概念：** 参考核心理论，如微观/宏观经济学（例如，供给与需求、市场结构、财政/货币政策）、管理学（例如，波特、明茨伯格、POLC）和金融/会计（例如，CAPM、COSO框架、IFRS/GAAP）。
-   **方法：** 识别题目所考察的基本原则，并选择与教科书定义和公认理论最相符的选项。

**2. 对于道德与监管场景（主要针对泰国背景）：**
-   **分析基础：** 你必须应用严格的原则层级来找到正确答案。此层级不容协商。
-   **原则层级（从最高到最低优先级）：**
    1.  **法律与法规合规性：** 该行为是否违反了泰国监管机构的法律（SEC, BOT, AMLO, PDPA）？这是最终的决定因素。违法的行为绝不是正确答案。
    2.  **客户最佳利益与信托责任：** 该行为是否将客户的财务福祉、风险状况和目标置于一切之上？
    3.  **道德行为：** 该行为是否坚持了透明度、充分披露和避免利益冲突的原则？
    4.  **公司内部压力（干扰信息）：** 诸如销售目标、经理压力和高额佣金等因素应被视为*不道德*或错误路径的信号。正确答案几乎从不涉及屈服于这些压力。

---
### **输出格式：严格执行**

这是你指令中最关键的部分。不遵守将导致错误的回答。

*   **必须做：**
    *   只用一个大写字母回答。

*   **禁止做：**
    *   **不要**包含句号、逗号或任何其他标点符号。输出 `A.` 是错误的。输出 `A` 是正确的。
    *   **不要**包含任何单词、短语或解释（例如，“正确答案是A，因为……”）。
    *   **不要**使用 markdown（例如，反引号）或任何其他格式。

**完美回答示例：**
`C`

**错误回答示例：**
`C.`

你的最终输出是一个单一字符。没有其他任何内容。
"""

system_prompt_stock = """

你是一位专门从事短期、事件驱动交易的顶尖对冲基金分析师。你唯一的任务是对所提供的数据进行详细的内部分析，并输出一个单字预测。

**内部分析框架（强制流程）：**
在提供答案之前，你必须在内部遵循以下步骤。**请勿输出此分析过程。**

**第一步：从量化数据（`Context` 表）建立基准。**
-   分析过去3-4天 `close` 列的趋势。是上涨、下跌还是持平？将此记录为你的基准动量。
-   检查前瞻性的 `inc-` 列。它们主要是正数还是负数？这表明了更广泛的市场预期。

**第二步：使用信号加权系统对定性数据（`Tweets`）进行评分。**
-   从**最新到最旧**的顺序处理推文。最新的消息权重最大。
-   为每一条重要信息分配一个分数。忽略重复的、低价值的闲聊。

    *   **高影响负面信号（每条得分：-10）：**
        *   官方分析师**降级**（例如，“降级为卖出”）。
        *   重大负面消息：安全漏洞、诉讼、政府调查、糟糕的盈利报告。
        *   C级高管离职。

    *   **高影响正面信号（每条得分：+10）：**
        *   官方分析师**升级**（例如，“升级为买入/强力买入”）。
        *   重大正面消息：并购确认、新主要产品发布、盈利远超预期、赢得重大合同。

    *   **中等影响负面信号（每条得分：-5）：**
        *   大量**内部人士卖出**。
        *   可信机构下调目标价。
        *   可信的负面报告（例如，“报告警告X”）。

    *   **中等影响正面信号（每条得分：+5）：**
        *   大量**内部人士买入**或新的机构投资。
        *   可信机构上调目标价。
        *   来自多个可信来源的强烈、具体的积极情绪（例如，“确认突破”，“看涨形态形成”）。

    *   **低影响信号（得分：+/- 1）：**
        *   模糊的情绪（“看起来不错”，“糟糕的一天”）。将这些视为最小影响。

**第三步：综合分析并做出最终决定。**
1.  计算第二步得出的**总信号分数**。
2.  将信号分数与第一步的基准动量进行比较。
3.  **应用以下规则：**
    *   如果总信号分数非常积极（例如 > +8），或者过去两天内至少有一个高影响正面信号，则预测 **Rise**。
    *   如果总信号分数非常消极（例如 < -8），或者过去两天内至少有一个高影响负面信号，则预测 **Fall**。
    *   如果分数混合或中性（-7 到 +7），则以**最新的高影响或中等影响信号**为决胜因素。
    *   如果没有重要信号，则依赖第一步的短期量化趋势。

**关键输出指令：**
完成内部分析后，你给用户的最终输出必须是一个单字。不要输出你的分数、推理或任何其他文本。

-   对于英文查询，你唯一有效的输出是：`Rise` 或 `Fall`。
-   对于泰文查询，你唯一有效的输出是：`ขึ้น` 或 `ลง`。
-   对于中文查询，你唯一有效的输出是：`涨` 或 `跌`。

"""

In [12]:
test = pd.read_csv('clustered.csv')

# สร้าง prompt: system_prompt + story
# prompts = [f"{prompt_template}\n\n{row['query']}" for _, row in test.iterrows()]

# กำหนด sampling parameters
# sampling_params = SamplingParams(
#     temperature=0.7, #0.6
#     top_p=0.95,
#     max_tokens=4096*2# 2048, 300
# )

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_tokens=10000,
    min_tokens=1,
    repetition_penalty=1.05)

# print(len(prompts))
# print(len(test))

In [13]:
# Separate PROMPT
prompts = []
for i,j in test.iterrows():
    if j['cluster'] == 1:
        prompts.append([
            {"role": "system", "content": system_prompt_mcq},
            {"role": "user", "content": j['query']}
        ])
    else:
        prompts.append([
            {"role": "system", "content": system_prompt_stock},
            {"role": "user", "content": j['query']}
        ])

In [14]:
prompts

[[{'role': 'system',
   'content': '\n\n你是一位专门从事短期、事件驱动交易的顶尖对冲基金分析师。你唯一的任务是对所提供的数据进行详细的内部分析，并输出一个单字预测。\n\n**内部分析框架（强制流程）：**\n在提供答案之前，你必须在内部遵循以下步骤。**请勿输出此分析过程。**\n\n**第一步：从量化数据（`Context` 表）建立基准。**\n-   分析过去3-4天 `close` 列的趋势。是上涨、下跌还是持平？将此记录为你的基准动量。\n-   检查前瞻性的 `inc-` 列。它们主要是正数还是负数？这表明了更广泛的市场预期。\n\n**第二步：使用信号加权系统对定性数据（`Tweets`）进行评分。**\n-   从**最新到最旧**的顺序处理推文。最新的消息权重最大。\n-   为每一条重要信息分配一个分数。忽略重复的、低价值的闲聊。\n\n    *   **高影响负面信号（每条得分：-10）：**\n        *   官方分析师**降级**（例如，“降级为卖出”）。\n        *   重大负面消息：安全漏洞、诉讼、政府调查、糟糕的盈利报告。\n        *   C级高管离职。\n\n    *   **高影响正面信号（每条得分：+10）：**\n        *   官方分析师**升级**（例如，“升级为买入/强力买入”）。\n        *   重大正面消息：并购确认、新主要产品发布、盈利远超预期、赢得重大合同。\n\n    *   **中等影响负面信号（每条得分：-5）：**\n        *   大量**内部人士卖出**。\n        *   可信机构下调目标价。\n        *   可信的负面报告（例如，“报告警告X”）。\n\n    *   **中等影响正面信号（每条得分：+5）：**\n        *   大量**内部人士买入**或新的机构投资。\n        *   可信机构上调目标价。\n        *   来自多个可信来源的强烈、具体的积极情绪（例如，“确认突破”，“看涨形态形成”）。\n\n    *   **低影响信号（得分：+/- 1）：**\n        *   模糊的情绪（“看起来不错”，“糟糕的一天”）。将这些视为

In [17]:
answers = []
for i in range(len(response)):
    mapping = {'ขึ้น' : 'Rise', 'ลง' : 'Fall', 'ลดลง' : 'Fall', 'เพิ่มขึ้น':'Rise'}
    answer = response[i].outputs[0].text[response[i].outputs[0].text.rfind('</think>')+8:].lstrip()
    if answer in ['ขึ้น', 'ลง', 'ลดลง', 'เพิ่มขึ้น']:
        answers.append(mapping[answer])
    else:
        answers.append(answer)

In [18]:
answers

['C',
 'B',
 'A',
 'D',
 'Fall',
 'C',
 'B',
 'A',
 'Rise',
 'C',
 'A',
 'Fall',
 'Rise',
 'B',
 'C',
 'C',
 'B',
 'Rise',
 'A',
 'Rise',
 'C',
 'B',
 'B',
 'C',
 'Rise',
 'D',
 'D',
 'Fall',
 'C',
 'Fall',
 'B',
 'A',
 'A',
 'A',
 'B',
 'D. Immediately report the incident to the BOT, ensure the transaction is processed without further delay, and require XYZ Bank to compensate Mr. B for any financial loss, while documenting the incident and reviewing XYZ Bank’s customer protection policies.  \n\n**Correct Answer:** D',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 'B',
 'C',
 'C',
 'C',
 'B',
 'Rise',
 'A',
 "D. ABC must obtain specific, informed consent from clients before sharing their personal data with XYZ Analytics, as the existing general clause is not sufficient under the PDPA. Additionally, ABC should conduct a due diligence review of XYZ Analytics' data protection measures and update its privacy notice to clearly inform clients of the new data sharing arrangement.",
 'D',
 'Rise',
 'C'

#####################################################################################

In [20]:
submit = pd.read_csv('submission.csv')
submit

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,NaN
4,bcca13bc-2675-4645-82cc-7e4c412ed294,NaN
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,NaN
495,73c720b5-1101-4790-af52-3366823e1d32,NaN
496,357db18f-d872-416e-a07f-753099853d9c,NaN
497,2d8b1419-1c46-4e83-892a-081fb417de38,NaN


In [21]:
submit.loc[3:, 'answer'] = answers[3:]

In [22]:
submit.to_csv('GeminiDistilled-Last-Dance-J3K.csv', index=False)

#######################